In [2]:
!pip3 install openai-whisper

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 798 kB 3.5 MB/s eta 0:00:01
  distutils: /private/var/folders/yp/vm433df166gcq6mdq6vchglh0000gn/T/pip-build-env-_k37bcyo/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/yp/vm433df166gcq6mdq6vchglh0000gn/T/pip-build-env-_k37bcyo/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/yp/vm433df166gcq6mdq6vchglh0000gn/T/pip-build-env-_k37bcyo/normal'
  distutils: /private/var/folders/yp/vm433df166gcq6mdq6vchglh0000gn/T/pip-build-env-_k37bcyo/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/yp/vm433df166gcq6mdq6vchglh0000gn/T/pip-build-env-_k37bcyo/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

In [7]:
# on Ubuntu or Debian
#sudo apt update && sudo apt install ffmpeg

# on Arch Linux
#sudo pacman -S ffmpeg

# on MacOS using Homebrew (https://brew.sh/)
!brew install ffmpeg

# on Windows using Chocolatey (https://chocolatey.org/)
#choco install ffmpeg

# on Windows using Scoop (https://scoop.sh/)
#scoop install ffmpeg

==> Downloading https://ghcr.io/v2/homebrew/core/ffmpeg/manifests/7.0.1
######################################################################### 100.0%
==> Fetching dependencies for ffmpeg: brotli, giflib, highway, imath, jpeg-turbo, libpng, xz, lz4, zstd, libtiff, little-cms2, openexr, webp, jpeg-xl, libvmaf, aom, aribb24, dav1d, freetype, fontconfig, frei0r, ca-certificates, gmp, libunistring, gettext, libidn2, libtasn1, nettle, p11-kit, openssl@3, libevent, libnghttp2, unbound, gnutls, pcre2, mpdecimal, readline, sqlite, python@3.12, glib, xorgproto, libxau, libxdmcp, libxcb, libx11, libxext, libxrender, lzo, pixman, cairo, graphite2, icu4c, harfbuzz, lame, fribidi, libunibreak, libass, libbluray, cjson, libmicrohttpd, mbedtls, librist, libsoxr, libssh, libvidstab, libogg, libvorbis, libvpx, opencore-amr, openjpeg, opus, rav1e, libsamplerate, flac, mpg123, libsndfile, rubberband, sdl2, snappy, speex, srt, svt-av1, leptonica, libb2, libarchive, pango, tesseract, theora, x264, x265, 

In [9]:
import whisper

whisper_model = whisper.load_model("base")

def transcribe_audio(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result['text']

# https://commons.wikimedia.org/wiki/File:Audio_Kevin_Folta.wav
# Audio example
audio_path = "./Audio_Kevin_Folta.wav" 

# Transcribir el audio
transcribed_text = transcribe_audio(audio_path)

print(transcribed_text)


/Users/sergiopaniegoblanco/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hi, my name is Kevin Fowler and I'm very grateful to have the opportunity to work in public science. I'm very fortunate to have been able to be trained by excellent mentors early in my career to give me a toolbox to be able to unravel important questions that can help us better understand our physical universe. I'm really excited about the opportunity to participate in the science that works around agriculture and find out ways that we can farm more sustainably, working in ways that we can help limit environmental impacts of farming, but also providing profitable ways for farmers to stay in business. We're really excited about the technologies that we create creating better foods for the American consumer and industrialized world consumer, but also getting to people in need so that they can have more selection of better fruits and vegetables. Going forward, I think that the future will depend upon the integration of all technologies at the table and it's really important that we pay a

/Users/sergiopaniegoblanco/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed Question:  Hi, my name is Kevin Fowler and I'm very grateful to have the opportunity to work in public science. I'm very fortunate to have been able to be trained by excellent mentors early in my career to give me a toolbox to be able to unravel important questions that can help us better understand our physical universe. I'm really excited about the opportunity to participate in the science that works around agriculture and find out ways that we can farm more sustainably, working in ways that we can help limit environmental impacts of farming, but also providing profitable ways for farmers to stay in business. We're really excited about the technologies that we create creating better foods for the American consumer and industrialized world consumer, but also getting to people in need so that they can have more selection of better fruits and vegetables. Going forward, I think that the future will depend upon the integration of all technologies at the table and it's really i

NameError: name 'prompt' is not defined

In [10]:
MODEL = "llama3"
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

/Users/sergiopaniegoblanco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, answer with "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
while True:
    print("Say 'exit' or 'quit' to exit the loop")
    question = input('User question: ')
    print(f"Question: {question}")
    if question.lower() in ["exit", "quit"]:
        print("Exiting the conversation. Goodbye!")
        break
    formatted_prompt = prompt.format(context=transcribed_text, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)
    print(f"Answer: {parsed_response}")
    print()

Say 'exit' or 'quit' to exit the loop
Question: What is his name?
Answer: Kevin Fowler!

Say 'exit' or 'quit' to exit the loop
Question: exit
Exiting the conversation. Goodbye!
